In [1]:
import pandas as pd
import torch
import lightning as L

from torch.utils.data import DataLoader
from lightning.pytorch.callbacks import ModelCheckpoint

from model.modeling_demolta import DeMOLTaConfig
from trainer import LitMOLA, get_mola_dataloader

In [2]:
torch.set_float32_matmul_precision('medium')

In [3]:
BATCH_SIZE = 4
MAX_STEP = 100000000
TEXT_MODEL_NAME = 'facebook/galactica-125m'

In [4]:
smiles_to_filter = pd.read_csv('./dataset/data/test.csv')['SMILES'].tolist()

In [5]:
train_dataloader = get_mola_dataloader(
    df_path='./preproc/pretrain.csv',
    ignore_smiles=smiles_to_filter,
    tokenizer_name=TEXT_MODEL_NAME,
    batch_size=BATCH_SIZE,
)

Using pad_token, but it is not set yet.
Using eos_token, but it is not set yet.


In [6]:
val_dataloader = get_mola_dataloader(
    df_path='./preproc/pretrain_val.csv',
    ignore_smiles=smiles_to_filter,
    tokenizer_name=TEXT_MODEL_NAME,
    batch_size=BATCH_SIZE,
)

Using pad_token, but it is not set yet.
Using eos_token, but it is not set yet.


In [7]:
demolta_config = DeMOLTaConfig(
    num_layers=12,
    hidden_dim=384,
    ff_dim=1536,
    num_heads=6,
)

In [8]:
lit_model = LitMOLA(
    demolta_config=demolta_config,
    text_model_name=TEXT_MODEL_NAME,
)

In [9]:
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='/.checkpoint/',
    filename='mola-pretrain-{step}-{val_loss:.2f}',
    save_top_k=3,
    every_n_train_steps=10
)

In [14]:
L.Trainer?

Init signature:
L.Trainer(
    *,
    accelerator: Union[str, lightning.pytorch.accelerators.accelerator.Accelerator] = 'auto',
    strategy: Union[str, lightning.pytorch.strategies.strategy.Strategy] = 'auto',
    devices: Union[List[int], str, int] = 'auto',
    num_nodes: int = 1,
    precision: Union[Literal[64, 32, 16], Literal['16-mixed', 'bf16-mixed', '32-true', '64-true'], Literal['64', '32', '16', 'bf16']] = '32-true',
    logger: Union[lightning.pytorch.loggers.logger.Logger, Iterable[lightning.pytorch.loggers.logger.Logger], bool, NoneType] = None,
    callbacks: Union[List[lightning.pytorch.callbacks.callback.Callback], lightning.pytorch.callbacks.callback.Callback, NoneType] = None,
    fast_dev_run: Union[int, bool] = False,
    max_epochs: Optional[int] = None,
    min_epochs: Optional[int] = None,
    max_steps: int = -1,
    min_steps: Optional[int] = None,
    max_time: Union[str, datetime.timedelta, Dict[str, int], NoneType] = None,
    limit_train_batches: Union[int

In [11]:
trainer = L.Trainer(
    accelerator='gpu',
    precision='16-mixed',
    max_steps=MAX_STEP,
    callbacks=[checkpoint_callback],
    accumulate_grad_batches=2,
    gradient_clip_val=1.0,
)


Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [12]:
trainer.fit(lit_model, train_dataloader, val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type  | Params
--------------------------------
0 | model | MOLLA | 48.5 M
--------------------------------
48.5 M    Trainable params
0         Non-trainable params
48.5 M    Total params
193.984   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

c:\Users\dust\anaconda3\envs\py310_torch2\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
c:\Users\dust\anaconda3\envs\py310_torch2\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
c:\Users\dust\anaconda3\envs\py310_torch2\lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:359: UserWarning: `ModelCheckpoint(monitor='val_loss')` could not find the monitored key in the returned metrics: ['train_loss', 'epoch', 'step']. HINT: Did you call `log('val_loss', value)` in the `LightningModule`?
  warning_cache.warn(m)
c:\Users\dust\anaconda3\envs\py310_torch2\lib\site-packages\lightning\pytorch\trainer\call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
